# Class #3

This time we'll build a natural number generator and an ASCII character generator.

---

## Using Caffeine

This is the only way I've found to use a library that's not in the Elixir core:
we navigate to the directory and then load the module.

In [1]:
File.cwd!() =~ "project/caffeine"

false

In [ ]:
cd("~/project/caffeine")

In [3]:
File.cwd!() =~ "project/caffeine"

true

In [4]:
Code.load_file("lib/caffeine.ex")

[{Caffeine.Element, <<70, 79, 82, 49, 0, 0, 3, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 130, 0, 0, 0, 12, 23, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 46, 69, 108, 101, 109, 101, 110, ...>>}, {Caffeine.Stream, <<70, 79, 82, 49, 0, 0, 17, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 19, 0, 0, 0, 31, 22, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 46, 83, 116, 114, 101, 97, ...>>}, {Caffeine, <<70, 79, 82, 49, 0, 0, 3, 220, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 122, 0, 0, 0, 12, 15, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 8, 95, 95, 105, 110, ...>>}]

Once we have Caffeine ready to use we change our definition of the LCG from last time to use Caffeine instead of the improvised stream we'd built (see the `stream/2` function). The `LCG.MMIX` module and the expressions imediately following are borrowed from last time.

In [5]:
defmodule LCG do
  defmodule Parameters do
    defstruct [
      modulus:    9,
      multiplier: 2,
      increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def instance(p) do
    m = p.modulus; a = p.multiplier; c = p.increment
    fn s when 0 <= s and s < m ->
      Integer.mod(a * s + c, m)
    end
  end

  def stream(seed, generator) do
    f = fn ->
      stream(generator.(seed), generator)
    end
    Caffeine.Stream.construct(seed, f)
  end
end

{:module, LCG, <<70, 79, 82, 49, 0, 0, 8, 56, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 27, 0, 0, 0, 28, 10, 69, 108, 105, 120, 105, 114, 46, 76, 67, 71, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, ...>>, {:stream, 2}}

In [6]:
defmodule LCG.MMIX do
  def instance do
    x = %LCG.Parameters{
      modulus:    round(:math.pow(2, 64)),
      multiplier: 6364136223846793005,
      increment:  1442695040888963407
    }
    true = LCG.Parameters.okay?(x)
    LCG.instance(x)
  end
end

{:module, LCG.MMIX, <<70, 79, 82, 49, 0, 0, 5, 160, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 175, 0, 0, 0, 17, 15, 69, 108, 105, 120, 105, 114, 46, 76, 67, 71, 46, 77, 77, 73, 88, 8, 95, 95, 105, 110, 102, 111, ...>>, {:instance, 0}}

In [7]:
m = LCG.MMIX.instance()

#Function<0.36907607/1 in LCG.instance/1>

In [8]:
Caffeine.Stream.take(LCG.stream(1, m), 9)

[1, 7806831264735756412, 9396908728118811419, 11960119808228829710, 7062582979898595269, 14673421054488193520, 9232803539723513983, 10218303843513747618, 1206773305466921929]

The LCG stream imposes on us large natural numbers. We have to build a way to work with them so that we can build data generators that are reasonable. I.e. we don't want to generate lists that are 7806831264735756412 elements long.

In [9]:
defmodule Generator.BoundNatural do
  def stream(limit: i) do
    s = LCG.stream(1, LCG.MMIX.instance())
    Caffeine.Stream.map(s, limit(i))
  end

  defp limit(i) do
    fn x ->
      Integer.mod(x, i)
    end
  end
end

{:module, Generator.BoundNatural, <<70, 79, 82, 49, 0, 0, 6, 36, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 247, 0, 0, 0, 22, 29, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 66, 111, 117, 110, 100, ...>>, {:limit, 1}}

In [10]:
x = Caffeine.Stream.take(Generator.BoundNatural.stream(limit: 100), 100)

[1, 12, 19, 10, 69, 20, 83, 18, 29, 44, 31, 54, 93, 48, 55, 58, 17, 52, 83, 46, 25, 24, 39, 46, 21, 48, 35, 86, 25, 48, 39, 38, 29, 72, 79, 38, 89, 28, 19, 78, 33, 48, 27, 22, 37, 76, 83, 98, 5, 28, ...]

Let's test that our distribution of _small_ natural numbers is uniform.

In [11]:
Enum.count(x, fn x -> x >= 50 end)

47

There's more than one way to write a stream of cube numbers (and other streams):
1. `Cube.V1` uses the primitives in Caffeine (like `construct/2`) and increments a counter
2. `Cube.V2` does away with the counter by `map`ing over the `Natural` stream

In [12]:
defmodule Cube.V1 do
  def stream do
    stream(0)
  end

  defp stream(x) do
    f = fn ->
      stream(x + 1)
    end
    Caffeine.Stream.construct(cube(x), f)
  end

  defp cube(x) do
    x * x * x
  end
end

{:module, Cube.V1, <<70, 79, 82, 49, 0, 0, 5, 204, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 187, 0, 0, 0, 19, 14, 69, 108, 105, 120, 105, 114, 46, 67, 117, 98, 101, 46, 86, 49, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:cube, 1}}

In [13]:
Caffeine.Stream.take(Cube.V1.stream(), 7)

[0, 1, 8, 27, 64, 125, 216]

In [14]:
defmodule Natural do
  def stream do
    stream(0)
  end

  defp stream(x) do
    f = fn ->
      stream(x + 1)
    end
    Caffeine.Stream.construct(x, f)
  end
end

{:module, Natural, <<70, 79, 82, 49, 0, 0, 5, 84, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 180, 0, 0, 0, 17, 14, 69, 108, 105, 120, 105, 114, 46, 78, 97, 116, 117, 114, 97, 108, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:stream, 1}}

In [15]:
Caffeine.Stream.take(Natural.stream(), 7)

[0, 1, 2, 3, 4, 5, 6]

In [16]:
defmodule Cube.V2 do
  def stream do
    Caffeine.Stream.map(Natural.stream(), &cube/1)
  end

  defp cube(x) do
    x * x * x
  end
end

{:module, Cube.V2, <<70, 79, 82, 49, 0, 0, 5, 100, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 194, 0, 0, 0, 19, 14, 69, 108, 105, 120, 105, 114, 46, 67, 117, 98, 101, 46, 86, 50, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:cube, 1}}

In [17]:
Caffeine.Stream.take(Cube.V2.stream(), 5)

[0, 1, 8, 27, 64]

This is **one** way to write a generator for printable ASCII characters.

In [18]:
defmodule Generator.Character.PrintableASCII do
  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 6, 164, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 14, 0, 0, 0, 23, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [19]:
Caffeine.Stream.take(Generator.Character.PrintableASCII.stream(), 96)

'!he@E@?l{$w@7$O:!2[:-LIBG@[&!^7:gp]f% W4S8qjgve,s@#$}vul?B;(GLGZ_J9Ha>!B%8#T]6q^iD[hY"kJM\\-8u4?,'

**One** way to write a generator for (ASCII) character lists.

In [20]:
defmodule Generator.CharacterList do
  alias Generator.BoundNatural
  alias Generator.Character.PrintableASCII

  def stream do
    stream(length: BoundNatural.stream(limit: 100), fill: PrintableASCII.stream())
  end

  defp stream(length: s, fill: r) do
    {x, y} = split(r, Caffeine.Stream.head(s))
    f = fn ->
      stream(length: Caffeine.Stream.tail(s), fill: y)
    end
    Caffeine.Stream.construct(x, f)
  end

  defp split(s, i) do
    {Caffeine.Stream.take(s, i), rest(s, i)}
  end

  defp rest(s, 0) do
    s
  end
  defp rest(s, i) when i > 0 do
    rest(Caffeine.Stream.tail(s), i - 1)
  end
end

{:module, Generator.CharacterList, <<70, 79, 82, 49, 0, 0, 8, 124, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 50, 0, 0, 0, 26, 30, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:rest, 2}}

In [21]:
x = Caffeine.Stream.take(Generator.CharacterList.stream(), 10)

['!', 'he@E@?l{$w@7', '$O:!2[:-LIBG@[&!^7:', 'gp]f% W4S8', 'qjgve,s@#$}vul?B;(GLGZ_J9Ha>!B%8#T]6q^iD[hY"kJM\\-8u4?,-J_:WV/L=Z]Pi$}', 'deD-.a>wR-@u\\O>/0c&I', 'tkh!h=rg2/N9r{81&}"74=^IVK>#<;<mBar#:CnC0m\\?6GLeXs$%:c\\_XWrs^/,[0i|SJa<q6)|= c&sp) ', 'G2aJ?*#RsPS"c0\'RGR', 'KVSN\'2i4!$SX;*7Re(/T{F-@]*ono', '`A&#$K8Q$kri\\Gp!hKHWZC6g6qb1v-L\'j_.?v%XCLE*A']

## Summary

Some of the things we've seen:

- Building streams with primitives (e.g. `Caffeine.Stream.{sentinel/0, construct/2}`
- Building streams with HOFs (e.g. `Caffeine.Stream.{map/2, filter/2}`)
- [Keyword lists](https://elixir-lang.org/getting-started/keywords-and-maps.html#keyword-lists):
The `Rectangle` example in [Smalltalk's Wikipedia article](https://en.wikipedia.org/wiki/Smalltalk#Messages) motivates their use for clarity at call site.

## Data Generators

- Natural

- Character

- Integer

- Float

- Boolean and `nil`

- List:

  A homogeneously typed list (elements are of the same type).
  API suggestion:
  `List.stream(g)` where `g` is any other generator.
  E.g. `List.stream(Integer.stream())` would build elements like `[1,8,0]`.


- Character List

- Pair (i.e. `[_|?]` where `?` isn't `[]`)

- Binary

- String

- Tuple:

  A heterogeneously typed tuple.
  API suggestion:
  `Tuple.stream(l)` where `l` is a list of generators.
  E.g. if `l = [Integer.stream(), Float.stream(), String.stream()]` then the elements would look like `{5, 6.1, "qwerty"}`.


- Record

- Symbol/Atom

- Map/Struct

- Function:

  This could be as simple as `Function.stream(arity: 0, return: Integer.stream())` with no restriction on the input parameters.
  Let's restrict arity to 5.
  E.g.:
  ```Elixir
  > f = head(Function.stream(arity: 1, return: String.stream()));
  > f.(nil)
  > "qwerty"
  ```
  
- Term:

  Every element produced by this generator is a value from a randomly selected generator.


- Port, PID, and Ref
